# Algorithm:

In [1]:
# Import requirement libraries
import pandas as pd
import numpy as np
from sklearn.metrics import *
import time
from datetime import timedelta

In [2]:
# Read results and the gold standard
cent_B = pd.read_csv('centralities_B.csv')
cent_C = pd.read_csv('centralities_C.csv')
cent_L = pd.read_csv('centralities_L.csv')

nodes_B = pd.read_csv('node_B.csv').drop_duplicates(ignore_index=True)
nodes_C = pd.read_csv('node_C.csv').drop_duplicates(ignore_index=True)
nodes_L = pd.read_csv('node_L.csv').drop_duplicates(ignore_index=True)
print(len(nodes_B))
print(len(nodes_C))
print(len(nodes_L))

nodes_L = pd.concat([nodes_L, pd.DataFrame(np.repeat(0, len(nodes_L)), columns = ['TRUE'])],axis=1)
nodes_B = pd.concat([nodes_B, pd.DataFrame(np.repeat(0, len(nodes_B)), columns = ['TRUE'])],axis=1)
nodes_C = pd.concat([nodes_C, pd.DataFrame(np.repeat(0, len(nodes_C)), columns = ['TRUE'])],axis=1)

driver_true = pd.read_csv('driver_true.csv')
driver_B = pd.concat([driver_true['Breast_cancer'].dropna(), 
                      pd.DataFrame(np.repeat(1, len(driver_true['Breast_cancer'].dropna())), columns=['TRUE'])], axis=1)
driver_B = driver_B.rename(columns={'Breast_cancer':'Name'})
driver_C = pd.concat([driver_true['Colon_cancer'].dropna(), 
                      pd.DataFrame(np.repeat(1, len(driver_true['Colon_cancer'].dropna())), columns=['TRUE'])], axis=1)
driver_C = driver_C.rename(columns={'Colon_cancer':'Name'})
driver_L = pd.concat([driver_true['Lung_cancer'].dropna(), 
                      pd.DataFrame(np.repeat(1, len(driver_true['Lung_cancer'].dropna())), columns=['TRUE'])], axis=1)
driver_L = driver_L.rename(columns={'Lung_cancer':'Name'})
print(driver_B.shape)
print(driver_C.shape)
print(driver_L.shape)

df_B = pd.read_excel("OtherMethods_B.xlsx")
df_C = pd.read_excel("OtherMethods_C.xlsx")
df_L = pd.read_excel("OtherMethods_L.xlsx")

10882
15664
11015
(572, 2)
(572, 2)
(566, 2)


In [3]:
def ALG(nodes, driver, cent, df, i, j, k):
    
    start_time = time.time()

    TN, FN, TP, FP, F_measure, acc, recall, precision, total_pred, DUW, DUN = [], [], [], [], [], [], [], [], [], [], []
    driver_pred_true = pd.DataFrame()
    driver_pred_total = pd.DataFrame()
    
    close_select = pd.DataFrame(cent.Name[cent['closeness centrality']> i]) 
    betweenness_select = pd.DataFrame(cent.Name[cent['betweenness centrality']> j])
    deg_select = pd.DataFrame(cent.Name[cent['degree centrality']> k])
    bt_deg = pd.DataFrame(set.intersection(set(betweenness_select.Name), set(deg_select.Name)), columns=['Name'])
    deg_close = pd.DataFrame(set.intersection(set(deg_select.Name), set(close_select.Name)), columns=['Name'])
    bt_close = pd.DataFrame(set.intersection(set(betweenness_select.Name), set(close_select.Name)), columns=['Name'])
    deg_close_bt = pd.DataFrame(set.intersection(set(deg_select.Name), 
                                                 set(close_select.Name), set(betweenness_select.Name)), columns=['Name'])
    intB2 = pd.concat([deg_close, bt_close, bt_deg]).drop_duplicates(ignore_index=True)
    inB2_3 = pd.merge(deg_close_bt, intB2, on ='Name', how ='outer').drop_duplicates(ignore_index=True)
    dy = pd.concat([pd.DataFrame(inB2_3), pd.DataFrame(np.repeat(1, len(inB2_3)), columns=['Label'])], axis=1)
    df1 = pd.merge(driver, dy, on ='Name', how ='outer')
    df1.fillna(0, inplace=True)
    df2 = pd.merge(df1, nodes, on =['Name','TRUE'], how ='outer')
    df2.fillna(0, inplace=True)
    yp = pd.merge(df2[df2.Label==1], df2[df2.TRUE==1], on =['Name','TRUE','Label'], how ='inner')
    driver_pred_true = pd.concat([driver_pred_true, yp.Name], axis=1)
    driver_pred_total = pd.concat([driver_pred_total, df2[df2.Label==1].Name], axis=1)
    confusionM = confusion_matrix(df2['TRUE'], df2['Label'])
    TN = np.append(TN, confusionM[1,1])
    FN = np.append(FN, confusionM[0,1])
    TP = np.append(TP, confusionM[0,0])
    FP = np.append(FP, confusionM[1,0])  
    acc = np.append(acc, accuracy_score(df2['TRUE'], df2['Label']))
    recall = np.append(recall, recall_score(df2['TRUE'], df2['Label']))
    precision = np.append(precision, precision_score(df2['TRUE'], df2['Label']))
    F_measure = np.append(F_measure, f1_score(df2['TRUE'], df2['Label']))
    DUW = np.append(DUW, len(set(yp['Name']) - set(df['iMaxDriver-W'])))
    DUN = np.append(DUN, len(set(yp['Name']) - set(df['iMaxDriver-N'])))
                
    elapsed_time_secs = time.time() - start_time
    msg = "Execution took: %s secs (clock time)" % timedelta(seconds=round(elapsed_time_secs)) 
    
    return(FN, TN, FP, TP, F_measure, acc, recall, precision, driver_pred_total, driver_pred_true, DUW, DUN, msg)

In [4]:
resultf_L = ALG(nodes_L, driver_L, cent_L, df_L, 1.95323008e-02, 1.45850935e-05, 1.49061274e-03)
resultf_L

(array([566.]),
 array([153.]),
 array([413.]),
 array([10449.]),
 array([0.2381323]),
 array([0.91546499]),
 array([0.27031802]),
 array([0.21279555]),
         Name
 4       PAX3
 9       SND1
 13      ETV5
 14     GATA2
 16      RARA
 ...      ...
 1127  YEATS4
 1128  HMG20A
 1129   PCBD1
 1130  POLR2B
 1131   ASCL1
 
 [719 rows x 1 columns],
       Name
 0     PAX3
 1     SND1
 2     ETV5
 3    GATA2
 4     RARA
 ..     ...
 148  PRDM1
 149  ZFHX3
 150  IKZF1
 151  MED12
 152    MYC
 
 [153 rows x 1 columns],
 array([38.]),
 array([74.]),
 'Execution took: 0:00:00 secs (clock time)')

In [5]:
resultf_B = ALG(nodes_B, driver_B, cent_B, df_B, 2.54740627e-01, 8.90329368e-06, 1.01903484e-03)
resultf_B

(array([633.]),
 array([164.]),
 array([408.]),
 array([10249.]),
 array([0.23959094]),
 array([0.90911472]),
 array([0.28671329]),
 array([0.20577164]),
         Name
 4       PAX3
 9       SND1
 13      ETV5
 14     GATA2
 16      RARA
 ...      ...
 1200    EYA3
 1201  RAD54B
 1202    HIRA
 1203   ESPL1
 1204   HCLS1
 
 [797 rows x 1 columns],
       Name
 0     PAX3
 1     SND1
 2     ETV5
 3    GATA2
 4     RARA
 ..     ...
 159  PRDM1
 160  ZFHX3
 161  IKZF1
 162  MED12
 163    MYC
 
 [164 rows x 1 columns],
 array([62.]),
 array([81.]),
 'Execution took: 0:00:00 secs (clock time)')

In [6]:
resultf_C = ALG(nodes_C, driver_C, cent_C, df_C, 0.24779346, 0.00030068, 0.00101107)
resultf_C

(array([590.]),
 array([158.]),
 array([414.]),
 array([15074.]),
 array([0.23939394]),
 array([0.93816211]),
 array([0.27622378]),
 array([0.21122995]),
          Name
 4        PAX3
 9        SND1
 13       ETV5
 14      GATA2
 16       RARA
 ...       ...
 1157     PAWR
 1158  NEUROD1
 1159   NFKBIA
 1160     MXD1
 1161    TCOF1
 
 [748 rows x 1 columns],
         Name
 0       PAX3
 1       SND1
 2       ETV5
 3      GATA2
 4       RARA
 ..       ...
 153  SMARCA4
 154    PRDM1
 155    ZFHX3
 156    IKZF1
 157      MYC
 
 [158 rows x 1 columns],
 array([52.]),
 array([76.]),
 'Execution took: 0:00:00 secs (clock time)')

In [7]:
driver_pred = pd.concat([resultf_L[8].rename(columns={'Name':'cMaxDriver_L'}),
                 resultf_B[8].rename(columns={'Name':'cMaxDriver_B'}),
                 resultf_C[8].rename(columns={'Name':'cMaxDriver_C'})],axis=1)
driver_pred = driver_pred.reset_index(drop=True)
driver_pred

,cMaxDriver_L,cMaxDriver_B,cMaxDriver_C
0,PAX3,PAX3,PAX3
1,SND1,SND1,SND1
2,ETV5,ETV5,ETV5
3,GATA2,GATA2,GATA2
4,RARA,RARA,RARA
...,...,...,...
908,NaN,NaN,FOXP1
909,NaN,NaN,HOXA13
910,NaN,NaN,SRSF3
911,NaN,NaN,EBF1


In [8]:
driver_pred.to_csv('driver_cMaxDriver.csv',index=False)

In [9]:
driver_pred_T = pd.concat([resultf_L[9].rename(columns={'Name':'cMaxDriver_L'}),
                 resultf_B[9].rename(columns={'Name':'cMaxDriver_B'}),
                 resultf_C[9].rename(columns={'Name':'cMaxDriver_C'})],axis=1)
driver_pred_T = driver_pred_T.reset_index(drop=True)
driver_pred_T

,cMaxDriver_L,cMaxDriver_B,cMaxDriver_C
0,PAX3,PAX3,PAX3
1,SND1,SND1,SND1
2,ETV5,ETV5,ETV5
3,GATA2,GATA2,GATA2
4,RARA,RARA,RARA
...,...,...,...
159,NaN,PRDM1,NaN
160,NaN,ZFHX3,NaN
161,NaN,IKZF1,NaN
162,NaN,MED12,NaN


In [10]:
driver_pred_T.to_csv('driver_true_cMaxDriver.csv',index=False)